# <center>Результаты модели переноса стиля *Adaptive Instance Normalization* в условиях инференса<center>

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import keras
import tensorflow as tf
import plotly.express as px
import warnings
# import cv2 as cv
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default = 'png'
pio.templates.default = 'plotly_dark'
IMAGES_FOLDER = '/mnt/s/CV/StyleTransferData/'
STYLE_FOLDER = os.path.join(IMAGES_FOLDER, 'wikiart_styles')

Возьмем одно изображение и 6 разных стилей

In [2]:
content = '/mnt/s/CV/StyleTransferData/test2015/COCO_test2015_000000011864.jpg'
styles = [
    '/mnt/s/CV/StyleTransferData/woman_with_hat_matisse.jpg',
    '/mnt/s/CV/StyleTransferData/wikiart/Cubism/albert-gleizes_femmes-cousant-1913.jpg',
    '/mnt/s/CV/StyleTransferData/wikiart/Abstract_Expressionism/walter-battiss_bosveld.jpg',
    '/mnt/s/CV/StyleTransferData/wikiart/Fauvism/adam-baltatu_house-on-siret-valley.jpg',
    '/mnt/s/CV/StyleTransferData/wikiart/Fauvism/alberto-magnelli_the-cafe-1914.jpg',
    '/mnt/s/CV/StyleTransferData/sketch.png'
]

Сохраним несколько изображений, используя разные чекпоинты обучения (каждый чекпоинт - отдельная эпоха)

Есть 5 чекпоинтов модели, у которой $\lambda=1$ (`model_{epoch}.keras`), также есть 5 чекпоинтов модели с $\lambda=20$ (`model_20_style_loss_{epoch}.keras.png`) 

Напишем функцию, чтобы сохранять полученные результаты в виде таблицы с тремя столбцами: контент, сгенерированное изображение, стиль

In [5]:
def save_outputs(model_names:list, save_content_colors:bool):
    for model_name in model_names:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            model = keras.models.load_model(os.path.join('checkpoint', model_name))
        model_name = model_name.rstrip('.keras')
        grid = make_subplots(
            rows=len(styles), 
            cols=3, 
            subplot_titles=['Content', 'Generated', 'Style'],
            x_title=model_name,
            horizontal_spacing=0.02, 
            vertical_spacing=0.02
        )
        iterators = [
            [content]*(len(styles)), 
            styles,
            (range(1, len(styles)+1)),
        ]
        for content_path, style_path, row in zip(*iterators):
            content_img = tf.expand_dims(tf.image.decode_image(tf.io.read_file(content_path)), 0)
            style_img = tf.expand_dims(tf.image.decode_image(tf.io.read_file(style_path)), 0)
            output = model.predict(content_img, style_img, alpha=1, save_content_colors=save_content_colors)
            grid.add_trace(
                px.imshow(content_img[0]).data[0],
                row=row,
                col=1
            ).add_trace(
                px.imshow(output[0]).data[0],
                row=row,
                col=2
            ).add_trace(
                px.imshow(style_img[0]).data[0],
                row=row,
                col=3
            ).update_layout(
                width=1400,
                height=len(styles)*600
            ).update_xaxes(
                visible=False
            ).update_yaxes(
                visible=False
            ).write_image(f'examples/{model_name}_{save_content_colors}.png')

In [6]:
save_outputs(os.listdir('checkpoint'), save_content_colors=False)

## <center>Первая эпоха
![](examples/style_model_1_False.png)

## <center>Пятая эпоха
![](examples/style_model_5_False.png)

Можно заметить, что лучше всего стиль передаёт последняя модель на пятой эпохе, у которой вес функции потерь стиля $\lambda=20$

Теперь сделаем сравнение последнего изображения с параметром `save_content_colors=True`. Этот параметр должен сохранять цвета оригинального изображения, перенося гистограмму контента на гистограмму стиля с помощью метода `match_histograms` из библиотеки `skimage`

In [7]:
save_outputs(model_names=['style_model_5.keras'], save_content_colors=True)

![](examples/style_model_5_True.png)

Больше всего заметна разница на третьем примере. Был черный рисунок со светлыми акцентами, а с параметром `save_content_colors=True` стал цветным. 

Теперь сравним влияние параметра `alpha`, отвечающего за интерполяцию между контентом и стилем. По-умолчанию `alpha=1`

In [8]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    model = keras.models.load_model('checkpoint/style_model_5.keras')
    model.training = False

In [9]:
alpha_thresh = tf.linspace(0.0, 1, 9).numpy()
grid = make_subplots(
    rows=3, 
    cols=3, 
    subplot_titles=['alpha='+str(alpha) for alpha in alpha_thresh],
    horizontal_spacing=1e-2,
    vertical_spacing=1e-1
)
content_img = tf.expand_dims(tf.io.decode_image(tf.io.read_file(content)), 0)
style_img = tf.expand_dims(tf.io.decode_image(tf.io.read_file(styles[-1])), 0)
iterators = [
    sorted(list(range(1, 4))*3),
    list(range(1, 4))*3,
    alpha_thresh
]
for row, col, alpha in zip(*iterators):
    output_img = model.predict(content_img, style_img, alpha=alpha)[0]
    grid.add_trace(
        px.imshow(output_img, title=str(alpha)).data[0],
        row=row,
        col=col
    )
grid.update_layout(
    width=1400,
    height=1400,
    title='<b>Сравнение выхода модели с разным уровнем <i>alpha</i></b>',
    title_x=0.5
).update_xaxes(
    visible=False
).update_yaxes(
    visible=False
).write_image('examples/alpha_thresholds.png')

![](examples/alpha_thresholds.png)